<a href="https://colab.research.google.com/github/tvani2/Neural-Networks-Facial-Expression-Recognition-Challenge/blob/main/Expr_recognition_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle wandb onnx -Uq

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! mkdir ~/.kaggle
!cp /content/drive/MyDrive/cs231n/assignments/assignment4/kaggle.json ~/.kaggle/kaggle.json
! chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c challenges-in-representation-learning-facial-expression-recognition-challenge
! unzip challenges-in-representation-learning-facial-expression-recognition-challenge

In [ ]:
import wandb
wandb.login()

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Load the dataset
# Change the file path to the correct location after unzipping
df = pd.read_csv('./train.csv')
X = df['pixels']
y = df['emotion']

train_size = 0.70
val_size = 0.15
test_size = 0.15
X_temp, X_test_new, y_temp, y_test_new = train_test_split(
    X, y, test_size=test_size, random_state=42, stratify=y
)
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=(val_size / (train_size + val_size)), random_state=42, stratify=y_temp
)

In [ ]:
from torch.utils.data import Dataset

def fast_process_pixels(pixel_series):
    pixel_lists = pixel_series.str.split()
    pixel_array = np.array(pixel_lists.tolist(), dtype=np.float32)
    return pixel_array.reshape(-1, 48, 48, 1) / 255.0

X_train_normalized = fast_process_pixels(X_train)
X_val_normalized = fast_process_pixels(X_val)
X_test_new_normalized = fast_process_pixels(X_test_new)

print("Data preprocessing completed!")
print(f"Train shape: {X_train_normalized.shape}")
print(f"Val shape: {X_val_normalized.shape}")
print(f"Test shape: {X_test_new_normalized.shape}")

# === 4. Dataset Class ===
class FastEmotionDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = torch.from_numpy(images).permute(0, 3, 1, 2).float()
        self.labels = torch.from_numpy(labels.values).long()
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

In [ ]:
import torch
from torchvision import transforms
from sklearn.utils.class_weight import compute_class_weight
from torch.utils.data import WeightedRandomSampler, DataLoader

# === 5. Transforms ===
train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomAffine(degrees=0, translate=(0.05, 0.05)),
    transforms.ColorJitter(brightness=0.1, contrast=0.1),
    transforms.Normalize([0.5], [0.5])
])

val_test_transforms = transforms.Compose([
    transforms.Normalize([0.5], [0.5])
])

# === 6. Create Datasets ===
train_dataset = FastEmotionDataset(X_train_normalized, y_train, transform=train_transforms)
val_dataset = FastEmotionDataset(X_val_normalized, y_val, transform=val_test_transforms)
test_dataset = FastEmotionDataset(X_test_new_normalized, y_test_new, transform=val_test_transforms)

# === 7. Compute Class Weights and Sampler ===
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = dict(zip(np.unique(y_train), class_weights))
sample_weights = np.array([class_weights_dict[label] for label in y_train])
sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)

# === 8. DataLoaders ===
batch_size = 64
num_workers = 0
pin_memory = torch.cuda.is_available()

train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler,
                          num_workers=num_workers, pin_memory=pin_memory,
                          persistent_workers=num_workers > 0)

val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False,
                        num_workers=num_workers, pin_memory=pin_memory,
                        persistent_workers=num_workers > 0)

test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False,
                         num_workers=num_workers, pin_memory=pin_memory,
                         persistent_workers=num_workers > 0)

In [ ]:
import gc

class EarlyStopping:
    def __init__(self, patience=7, min_delta=0.001, restore_best_weights=True):
        self.patience = patience
        self.min_delta = min_delta
        self.restore_best_weights = restore_best_weights
        self.best_loss = float('inf')
        self.counter = 0
        self.best_weights = None

    def __call__(self, val_loss, model):
        if val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
            if self.restore_best_weights:
                self.best_weights = model.state_dict().copy()
        else:
            self.counter += 1

        if self.counter >= self.patience:
            if self.restore_best_weights and self.best_weights:
                model.load_state_dict(self.best_weights)
            return True
        return False

gc.collect()
print("Pipeline is ready for model training!")

In [ ]:
class EmotionDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        # Convert to torch tensors and rearrange dimensions (H, W, C) -> (C, H, W)
        self.images = torch.FloatTensor(images).permute(0, 3, 1, 2)
        self.labels = torch.LongTensor(labels.values)
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

# Define data augmentation transforms
train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=10),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)),
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize to [-1, 1]
])

# No augmentation for validation and test
val_test_transforms = transforms.Compose([
    transforms.Normalize(mean=[0.5], std=[0.5])  # Same normalization as training
])

# Create datasets
train_dataset = EmotionDataset(X_train_normalized, y_train, transform=train_transforms)
val_dataset = EmotionDataset(X_val_normalized, y_val, transform=val_test_transforms)
test_dataset = EmotionDataset(X_test_new_normalized, y_test_new, transform=val_test_transforms)

In [ ]:
class_weights = compute_class_weight(
    'balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weights_dict = {i: class_weights[i] for i in range(len(class_weights))}

# Create sample weights for each training sample
sample_weights = [class_weights_dict[label] for label in y_train]
sampler = WeightedRandomSampler(
    weights=sample_weights,
    num_samples=len(sample_weights),
    replacement=True
)

In [ ]:
# Create DataLoaders
batch_size = 64
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_new_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Print class distribution info
print("Class distribution in training set:")
print(y_train.value_counts().sort_index())
print(f"\nClass weights: {class_weights_dict}")

In [ ]:
import wandb

wandb.init(project="Facial_Expression_Recognition_1", name="EmotionCNN_Run1")

wandb.config.update({
    "architecture": "EmotionCNN",
    "dropout_feature": 0.25,
    "dropout_classifier": 0.5,
    "scheduler": "ReduceLROnPlateau",
    "optimizer": "Adam",
    "scheduler_patience": 7,
    "gradient_clip_norm": 1.0,
    "early_stopping_patience": 7,
    "input_size": (48, 48),
    "num_classes": 7,
    "batch_norm": True,
    "batch_size": 64,
    "epochs": 20,
    "learning_rate": 0.001,
    "weight_decay" : 1e-5,
})

raw_data_artifact = wandb.Artifact(
    name="facial-expression-dataset",
    type="dataset",
    description="Facial Expression Recognition Challenge Dataset"
)
# Log the artifact to the current run
wandb.log_artifact(raw_data_artifact)

print("Data loaded and logged as a wandb artifact.")

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class EmotionCNN(nn.Module):
    def __init__(self, num_classes=7):
        super(EmotionCNN, self).__init__()

        self.features = nn.Sequential(
            # Block 1
            nn.Conv2d(1, 64, kernel_size=3, padding=1),  # (1, 48, 48) -> (64, 48, 48)
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),  # -> (64, 24, 24)

            # Block 2
            nn.Conv2d(64, 128, kernel_size=3, padding=1),  # -> (128, 24, 24)
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),  # -> (128, 12, 12)

            # Block 3
            nn.Conv2d(128, 256, kernel_size=3, padding=1),  # -> (256, 12, 12)
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),  # -> (256, 6, 6)
            nn.Dropout(0.25)
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),  # -> (256*6*6)
            nn.Linear(256 * 6 * 6, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, ConcatDataset, Dataset, WeightedRandomSampler # Ensure these are imported
from torchvision import transforms # Ensure transforms is imported
from sklearn.metrics import accuracy_score
import numpy as np

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = EmotionCNN(num_classes=7).to(device)
wandb.watch(model, log="all")
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)

# Track loss and accuracy
train_losses, train_accuracies = [], []
val_losses = []
val_accuracies = []

# Add scheduler and early stopping
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.1, verbose=True)
early_stopping = EarlyStopping(patience=7, min_delta=0.001)

# Training loop
epochs = 20
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    # Use the pre-defined train_loader
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        total_train += labels.size(0)
        correct_train += predicted.eq(labels).sum().item()

    avg_train_loss = running_loss / len(train_loader.dataset)
    train_accuracy = 100. * correct_train / total_train
    train_losses.append(avg_train_loss)
    train_accuracies.append(train_accuracy)

    # Validation loop
    model.eval()
    running_val_loss = 0.0
    correct_val = 0
    total_val = 0
    all_val_preds, all_val_labels = [], []

    with torch.no_grad():
        # Use the pre-defined val_loader
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            val_loss = criterion(outputs, labels)

            running_val_loss += val_loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            total_val += labels.size(0)
            correct_val += predicted.eq(labels).sum().item()
            all_val_preds.extend(predicted.cpu().numpy())
            all_val_labels.extend(labels.cpu().numpy())

    avg_val_loss = running_val_loss / len(val_loader.dataset)
    val_accuracy = 100. * correct_val / total_val
    val_losses.append(avg_val_loss)
    val_accuracies.append(val_accuracy)

    print(f"Epoch {epoch+1:02d}: Train Loss = {avg_train_loss:.4f}, Train Acc = {train_accuracy:.2f}%, "
          f"Val Loss = {avg_val_loss:.4f}, Val Acc = {val_accuracy:.2f}%")

    # Log metrics to wandb
    wandb.log({
        "epoch": epoch + 1,
        "train_loss": avg_train_loss,
        "train_accuracy": train_accuracy,
        "val_loss": avg_val_loss,
        "val_accuracy": val_accuracy,
        "learning_rate": optimizer.param_groups[0]['lr']
    })

    # Step the scheduler
    scheduler.step(avg_val_loss)

    # Check for early stopping
    if early_stopping(avg_val_loss, model):
        print(f"Early stopping at epoch {epoch+1}")
        break

# Evaluate on test set (using the pre-defined test_loader)
print("\nEvaluating on test set...")
model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = outputs.max(1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

test_accuracy = accuracy_score(all_labels, all_preds) * 100
print(f"\n✅ Test Accuracy: {test_accuracy:.2f}%")

# Log final test accuracy to wandb
wandb.log({"test_accuracy": test_accuracy})

# Finish the wandb run
wandb.finish()

In [26]:
# Define EmotionCNN
class EmotionCNN(nn.Module):
    def __init__(self, num_classes=7):
        super(EmotionCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout(0.25)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * 6 * 6, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

In [24]:
wandb.init(project="Facial_Expression_Recognition_1", name="EmotionCNN_Run1")

wandb.config.update({
    "architecture": "EmotionCNN",
    "dropout_feature": 0.25,
    "dropout_classifier": 0.5,
    "scheduler": "ReduceLROnPlateau",
    "optimizer": "Adam",
    "scheduler_patience": 5,
    "gradient_clip_norm": 1.0,
    "early_stopping_patience": 10,
    "input_size": (48, 48),
    "num_classes": 7,
    "batch_norm": True,
    "batch_size": 64,
    "epochs": 50,
    "learning_rate": 0.0005,
    "weight_decay": 1e-5,
})

# Create a wandb artifact for the dataset
raw_data_artifact = wandb.Artifact(
    name="facial-expression-dataset",
    type="dataset",
    description="Facial Expression Recognition Challenge Dataset"
)
# Log the artifact to the current run
wandb.log_artifact(raw_data_artifact)

print("Data loaded and logged as a wandb artifact.")

Data loaded and logged as a wandb artifact.


In [28]:
class EarlyStopping:
    def __init__(self, patience=7, min_delta=0.001, restore_best_weights=True):
        self.patience = patience
        self.min_delta = min_delta
        self.restore_best_weights = restore_best_weights
        self.best_loss = float('inf')
        self.counter = 0
        self.best_weights = None

    def __call__(self, val_loss, model):
        if val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
            if self.restore_best_weights:
                self.best_weights = model.state_dict().copy()
        else:
            self.counter += 1

        if self.counter >= self.patience:
            if self.restore_best_weights and self.best_weights:
                model.load_state_dict(self.best_weights)
            return True
        return False

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader # Ensure DataLoader is imported
from sklearn.metrics import accuracy_score
import numpy as np
import os
from torch.utils.data import Dataset, WeightedRandomSampler # Ensure these are imported
from torchvision import transforms # Ensure transforms is imported
from sklearn.utils.class_weight import compute_class_weight # Ensure this is imported
import gc # Ensure gc is imported for garbage collection

try:
    class_weights = compute_class_weight(
        'balanced',
        classes=np.unique(y_train),
        y=y_train
    )
    class_weights_dict = {i: class_weights[i] for i in range(len(class_weights))}
    sample_weights = [class_weights_dict[label] for label in y_train]
    sampler = WeightedRandomSampler(
        weights=sample_weights,
        num_samples=len(sample_weights),
        replacement=True
    )

    # Create DataLoaders
    batch_size = 64
    num_workers = 0
    train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler, num_workers=num_workers)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

    # Print class distribution info
    print("Class distribution in training set:")
    print(y_train.value_counts().sort_index())
    print(f"\nClass weights: {class_weights_dict}")
except NameError:
    print("Error: y_train, train_dataset, val_dataset, or test_dataset not defined. Please load your data.")

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize model
model = EmotionCNN(num_classes=7).to(device)
wandb.watch(model, log="all")  # Track model gradients and parameters

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005, weight_decay=1e-5)

# Track metrics
train_losses, train_accuracies = [], []
val_losses, val_accuracies = [], []

# Scheduler and early stopping
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.5, verbose=True)
early_stopping = EarlyStopping(patience=10, min_delta=0.0005)

# Save best model
best_val_loss = float('inf')
model_save_path = "best_model.pth"

epochs = 29
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        total_train += labels.size(0)
        correct_train += predicted.eq(labels).sum().item()

    avg_train_loss = running_loss / len(train_loader.dataset)
    train_accuracy = 100. * correct_train / total_train
    train_losses.append(avg_train_loss)
    train_accuracies.append(train_accuracy)

    # Validation
    model.eval()
    running_val_loss = 0.0
    correct_val = 0
    total_val = 0
    all_val_preds, all_val_labels = [], []

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            val_loss = criterion(outputs, labels)

            running_val_loss += val_loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            total_val += labels.size(0)
            correct_val += predicted.eq(labels).sum().item()
            all_val_preds.extend(predicted.cpu().numpy())
            all_val_labels.extend(labels.cpu().numpy())

    avg_val_loss = running_val_loss / len(val_loader.dataset)
    val_accuracy = 100. * correct_val / total_val
    val_losses.append(avg_val_loss)
    val_accuracies.append(val_accuracy)

    # Log training and validation metrics to wandb
    wandb.log({
        "epoch": epoch + 1,
        "train_loss": avg_train_loss,
        "train_accuracy": train_accuracy,
        "val_loss": avg_val_loss,
        "val_accuracy": val_accuracy,
        "learning_rate": optimizer.param_groups[0]['lr']
    })

    print(f"Epoch {epoch+1:02d}: Train Loss = {avg_train_loss:.4f}, Train Acc = {train_accuracy:.2f}%, "
          f"Val Loss = {avg_val_loss:.4f}, Val Acc = {val_accuracy:.2f}%")

    scheduler.step(avg_val_loss)

    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), model_save_path)
        # Log the best model as a wandb artifact
        model_artifact = wandb.Artifact(
            name=f"best-model-epoch-{epoch+1}",
            type="model",
            description="Best EmotionCNN model based on validation loss"
        )
        model_artifact.add_file(model_save_path)
        wandb.log_artifact(model_artifact)

    if early_stopping(avg_val_loss, model):
        print(f"Early stopping at epoch {epoch+1}")
        break

# Load best model before final test
model.load_state_dict(torch.load(model_save_path))

# Final Evaluation
print("\nEvaluating on test set...")
model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = outputs.max(1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

test_accuracy = accuracy_score(all_labels, all_preds) * 100
print(f"\n✅ Test Accuracy: {test_accuracy:.2f}%")

# Log test accuracy to wandb
wandb.log({"test_accuracy": test_accuracy})

# Finish the wandb run
wandb.finish()

Class distribution in training set:
emotion
0    2797
1     306
2    2867
3    5051
4    3380
5    2219
6    3475
Name: count, dtype: int64

Class weights: {0: np.float64(1.0263547678635272), 1: np.float64(9.381419234360411), 2: np.float64(1.0012955304200508), 3: np.float64(0.568345730689821), 4: np.float64(0.849323753169907), 5: np.float64(1.2936972896414085), 6: np.float64(0.8261048304213772)}


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 01: Train Loss = 1.9714, Train Acc = 17.06%, Val Loss = 1.8675, Val Acc = 21.50%
Epoch 02: Train Loss = 1.8906, Train Acc = 20.22%, Val Loss = 1.8286, Val Acc = 22.57%
Epoch 03: Train Loss = 1.8584, Train Acc = 22.36%, Val Loss = 1.7610, Val Acc = 33.36%
Epoch 04: Train Loss = 1.8251, Train Acc = 23.71%, Val Loss = 1.9802, Val Acc = 25.89%
Epoch 05: Train Loss = 1.7988, Train Acc = 25.83%, Val Loss = 1.7045, Val Acc = 34.87%
Epoch 06: Train Loss = 1.7696, Train Acc = 27.13%, Val Loss = 1.7095, Val Acc = 29.79%
Epoch 07: Train Loss = 1.7575, Train Acc = 28.18%, Val Loss = 1.6222, Val Acc = 37.15%
Epoch 08: Train Loss = 1.7438, Train Acc = 28.68%, Val Loss = 1.6289, Val Acc = 38.01%
Epoch 09: Train Loss = 1.7332, Train Acc = 29.25%, Val Loss = 1.6128, Val Acc = 37.80%
Epoch 10: Train Loss = 1.7171, Train Acc = 29.88%, Val Loss = 1.6035, Val Acc = 41.54%
Epoch 11: Train Loss = 1.7119, Train Acc = 30.54%, Val Loss = 1.5664, Val Acc = 41.98%
Epoch 12: Train Loss = 1.7066, Train Acc = 